# ASAXS data reduction

some script description

## Libraries

In [2]:
import pandas as pd
import os
import csv
import time

import numpy as np
from scipy import optimize
import scipy.constants as con
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, LinearSegmentedColormap

import fabio
import pyFAI
from pyFAI.detectors import Pilatus1M

## Read the basic data

In [20]:
# give the directory
data_dir = 'files\Data_for_ASAXS'
# name if data file ontaining all nessesary information
metaFileName = 'Data_for_ASAXS_ASCII_out_changed.txt'

#read the meta file
path_to_metaFile = data_dir + os.sep + metaFileName
metaFrame = pd.read_csv(path_to_metaFile, sep='\,',index_col=0,engine='python')

In [36]:
tiffName = metaFrame['Pilatus_filename'][0].split('/')[-1]
tiffPath = data_dir + os.sep + tiffName
energy = metaFrame['Energy'][0]
print(tiffPath, energy)

files\Data_for_ASAXS\pilatus_2020kw29tg14_00378.tif 7000.002692
